In [1]:
import numpy as np
import pandas as pd

In [4]:
books=pd.read_csv('Books.csv', error_bad_lines=False, encoding='latin-1') 

c:\users\abhijith\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [7]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

In [9]:
books.rename(columns={'Book-Title': 'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'}, inplace=True)

In [10]:
books.head()

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [11]:
users = pd.read_csv('Users.csv', error_bad_lines=False, encoding='latin-1')

In [12]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [13]:
users.rename(columns={'User-ID':'user_id', 'Location':'location', 'Age':'age'}, inplace=True)

In [14]:
users.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [15]:
 ratings = pd.read_csv('Ratings.csv', error_bad_lines=False, encoding='latin-1')

In [16]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [17]:
ratings.rename(columns={'User-ID':'user_id', 'Book-Rating':'rating'}, inplace=True)

In [18]:
ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [19]:
books.shape

(271360, 5)

In [20]:
users.shape

(278858, 3)

In [22]:
ratings.shape

(1149780, 3)

In [23]:
x=ratings['user_id'].value_counts()>200

In [24]:
y = x[x].index

In [25]:
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            116122,  28634, 188951,  59727, 155916, 274808,  73681,   9856,
            268622,  44296],
           dtype='int64', length=899)

In [26]:
ratings=ratings[ratings['user_id'].isin(y)]

In [27]:
ratings.shape

(526356, 3)

In [28]:
ratings_with_books=ratings.merge(books, on='ISBN')

In [30]:
ratings_with_books.shape

(487671, 7)

In [31]:
number_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [32]:
number_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [33]:
number_rating.rename(columns={'rating':'number_of_ratings'}, inplace=True)

In [35]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [37]:
final_rating.shape

(487671, 8)

In [38]:
final_rating = final_rating[final_rating['number_of_ratings']>=50]

In [39]:
final_rating.shape

(61853, 8)

In [40]:
final_rating.drop_duplicates(['user_id', 'title'], inplace=True)

<ipython-input-40-74cc5615107c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating.drop_duplicates(['user_id', 'title'], inplace=True)


In [41]:
final_rating.shape

(59850, 8)

In [42]:
book_pivot=final_rating.pivot_table(columns='user_id', index='title', values='rating')

In [45]:
book_pivot.fillna(0,inplace=True)

In [48]:
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(book_pivot)

In [49]:
type(book_sparse)

scipy.sparse.csr.csr_matrix

In [51]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [52]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [53]:
distances, suggestions=model.kneighbors(book_pivot.iloc[237, :].values.reshape(1,-1), n_neighbors=6)

In [54]:
distances

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [55]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [66]:
def recommend_book(book_name):
    book_id=np.where(book_pivot.index==book_name)[0][0]
    distances, suggestions=model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1,-1), n_neighbors=6)
    
    for i in range(len(suggestions[0])):
        if i == 0:
            print("The Suggestions for", book_name, "are: ")
        else:    
            print(book_pivot.index[suggestions[0][i]])

In [67]:
recommend_book('Animal Farm')

The Suggestions for Animal Farm are: 
Exclusive
Jacob Have I Loved
Second Nature
Pleading Guilty
No Safe Place
